# **Importación de paquetes**
---

In [43]:
import pandas as pd
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.preprocessing import OrdinalEncoder

# **Carga de los datos**
---

In [44]:
ruta_dataset='../../Dataset_Apart/Raw/DatasetAirbnb_v1.csv'
data=pd.read_csv(ruta_dataset)

# **Extracción de los datos de Informacion y Limpieza evaluaciones**
---

In [45]:
import re
patt=r'([0-9]+\xa0viajeros)'
patt_hue=r'([0-9]+\xa0huéspedes)'
patt_camas=r'([0-9]+\scama[s]?)'
patt_ban=r'([0-9]+\sbaño[s]?)'
patt_drom=r'([0-9]+\sdormitorio[s]?)'

#Procesamiento del dataframe
#-----
data['Evaluaciones']=data['Evaluaciones'].str.replace(' evaluaciones','').str.replace('\xa0evaluaciones','').astype(float)
data['Precio']=data['Precio'].str.replace('€','').astype(float)
data['Limpieza']=data['Limpieza'].str.replace('Limpieza','').str.replace(',','.').astype(float)
data['Veracidad']=data['Veracidad'].str.replace('Veracidad','').str.replace(',','.').astype(float)
data['Llegada']=data['Llegada'].str.replace('Llegada','').str.replace(',','.').astype(float)
data['Comunicacion']=data['Comunicacion'].str.replace('Comunicación','').str.replace(',','.').astype(float)
data['Ubicacion']=data['Ubicacion'].str.replace('Ubicación','').str.replace(',','.').astype(float)
data['Calidad']=data['Calidad'].str.replace('Calidad','').str.replace(',','.').astype(float)

data['Capacidad']=data['Informacion'].apply(lambda x:re.findall(patt,x)[0] if len(re.findall(patt,x))>0 else re.findall(patt_hue,x)[0] if len(re.findall(patt_hue,x))>0 else np.nan)
data['Capacidad']=data['Capacidad'].str.replace('\xa0viajeros','').str.replace('\xa0huéspedes','').astype(float)
data['Camas']=data['Informacion'].apply(lambda x: re.findall(patt_camas,x)[0] if len(re.findall(patt_camas,x)) else np.nan)
data['Camas']=data['Camas'].str.replace(' camas','').str.replace(' cama','').astype(float)
data['Baños']=data['Informacion'].apply(lambda x: re.findall(patt_ban,x)[0] if len(re.findall(patt_ban,x)) else np.nan)
data['Baños']=data['Baños'].str.replace(' baños','').str.replace(' baño','').str.replace('\xa0baños','').astype(float)
data['Dormitorios']=data['Informacion'].apply(lambda x: re.findall(patt_drom,x)[0] if len(re.findall(patt_drom,x)) else np.nan)
data['Dormitorios']=data['Dormitorios'].str.replace(' dormitorios','').str.replace(' dormitorio','').astype(float)
data['Baño Compartido']=data['Informacion'].apply(lambda x: 1 if x.find('Baño compartido')!=-1 else 0)

#Eliminamos la columna información
data.drop('Informacion',axis=1,inplace=True)

# **Extracción de latitud y longitud**
---

In [46]:
from geopy.geocoders import Nominatim

def lat_log(l):
    geolocator = Nominatim(user_agent="España")
    try:
        location = geolocator.geocode(l)
        latitude = location.latitude
        longitude = location.longitude
        return latitude, longitude
    except:
        return np.nan,np.nan
data['Latitud_Longitud']=data['Localizacion'].apply(lambda x: lat_log(x))
data['Latitud']=data['Latitud_Longitud'].apply(lambda x: x[0])
data['Longitud']=data['Latitud_Longitud'].apply(lambda x: x[1])
data.drop('Latitud_Longitud',axis=1,inplace=True)

# **Extracción del País de localización**
---

In [47]:
def ubi(x):
    try:
        dic={}
        e=x.find('España') if x.find('España')!=-1 else 1e10
        p=x.find('Portugal') if x.find('Portugal')!=-1 else 1e10
        f=x.find('Francia') if x.find('Francia')!=-1 else 1e10
        dic[e]='España'
        dic[p]='Portugal'
        dic[f]='Francia'
        if e==p==f:
            return np.nan
        else:
            minimo=min(e,p,f)
            return x.split(dic[minimo])[0] + dic[minimo]
    except:
        return np.nan

data['Pais']=data['Localizacion'].apply(lambda x: str(x).split(',')[-1].replace(' ',''))
data['Pais']=data['Pais'].apply(lambda x: x if x in ['España','Portugal','Francia'] else np.nan)
data.drop('Localizacion',axis=1,inplace=True)

# **Extracción de los servicios**
---

In [48]:
data['Wifi']=data['Servicios'].str.contains('Wifi').astype(str)
data['Mascotas']=data['Servicios'].str.contains('Admite mascotas').astype(str)
data['Piscina']=data['Servicios'].str.contains('Piscina').astype(str)
data['Parking']=data['Servicios'].str.contains('Aparcamiento gratuito').astype(str)
data.drop('Servicios',axis=1,inplace=True)

# **Eliminación de camas y dormitorios**
---
Elevada correlación con capacidad.

In [49]:
data.drop(['Camas','Dormitorios'],axis=1,inplace=True)

# **Imputación de nulos categóricos por la más frecuente**
---

In [50]:
cat_cols_aplican=data[[col for col in data.columns if col not in ['Titulo','Descripcion Simple','URL']]].select_dtypes(include='object').columns
cat_imputer=SimpleImputer(strategy='most_frequent')
data[cat_cols_aplican]=cat_imputer.fit_transform(data[cat_cols_aplican])

# **Imputación de nulos por la mediana**
---

In [51]:
num_cols_aplican=data.select_dtypes(exclude='object').columns
num_imputer=SimpleImputer(strategy='median')
data[num_cols_aplican]=num_imputer.fit_transform(data[num_cols_aplican])

# **Precio por persona por noche**
---

In [52]:
data['Precio_por_persona']=data['Precio']/data['Capacidad']

# **Transformación logarítmica del precio**
---

In [53]:
data['Precio']=np.log(data['Precio'])

# **Cálculo de Rating**
---

In [54]:
data['Rating']=data[['Limpieza','Veracidad','Llegada','Comunicacion','Ubicacion','Calidad']].mean(axis=1)

# **Conversión tipos**
---

In [58]:
#Servicios
data['Wifi']=data['Wifi'].astype(bool).astype(float)
data['Mascotas']=data['Mascotas'].astype(bool).astype(float)
data['Piscina']=data['Piscina'].astype(bool).astype(float)
data['Parking']=data['Parking'].astype(bool).astype(float)

#Label Encoder
encoder=OrdinalEncoder()
cols=['Pais','Tipo',]
data[cols]=encoder.fit_transform(data[cols])

# **Guardado del dataset Limpio**
---

In [61]:
data.to_csv(f'DatasetAirbnb_Cleaned_v1.csv',index=False)

Ya podemos hacer la selección de variables.